In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM"
!ls

/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM
architecture.png	  dataset.py	  logs	      models		preprocess.py  train.py
china_pm_data.csv	  impute.ipynb	  main.ipynb  model_test.ipynb	__pycache__    utils.py
china_pm_data_nonull.csv  interpolate.py  model.py    permutation.py	test.ipynb


In [3]:
from tqdm import tqdm
from utils import set_logger, prepare_data_for_CNN_LSTM
from dataset import FinedustCNNLSTMDataset
from model import FinedustCNNLSTM
from utils import prepare_data
from sklearn.model_selection import train_test_split
import os
import logging
from interpolate import simple_interpolate
import numpy as np
from preprocess import minmax_scaling
from train import train, train_cnn_lstm
from permutation import compute_cnn_lstm_permutation_importance

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

In [15]:
config = {
    "learning_rate": 1e-4,
    "epochs": 500,
    "batch_size": 32,
    "num_layers": 2,
    "hidden_size": 128,
    "window_size": 24,
    "output_size": 1,
    "dropout_prob": 0.3,
    "patience": 10,
    "out_channels": 4,
    "kernel_size": 3,
    "K": 3,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CNN-LSTM

Model
* Jeonju: `Jeonju_K=2_lr=0.0001.pth`
* Seoul: `Seoul_K=2_lr=0.0001_p=0.2_hidden_size=256.pth`
* Gwangju: `Gwangju_K=1.pth`
* Daeju: `Daegu_K=1.pth`
* Andong: `Andong_K=1.pth`

In [19]:
config = {
    "learning_rate": 1e-4,
    "epochs": 500,
    "batch_size": 32,
    "num_layers": 2,
    "hidden_size": 256,
    "window_size": 24,
    "output_size": 1,
    "dropout_prob": 0.2,
    "patience": 10,
    "out_channels": 6,
    "kernel_size": 3,
    "K": 2,
}

regions = ["Seoul"]

pred_list, loss_list = [], []

for region in regions:
  if region == "Jeonju":
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "IX", "PS", "지점", "위도", "경도"]
  else:
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]
  df = prepare_data_for_CNN_LSTM(region.lower(), columns_to_remove)
  df = simple_interpolate(df, method="linear")
  df, scaler = minmax_scaling(df)

  dataset = FinedustCNNLSTMDataset(df,
                                  window_size=config["window_size"],
                                  prediction_length=config["output_size"],
                                  time_window=3)
  train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
  train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

  model = FinedustCNNLSTM(config,
                          in_channels=len(dataset.pm_columns),
                          input_size=len(dataset.feature_columns)).to(device)

  total_preds, losses = train_cnn_lstm(model,
                                      train_dataset,
                                      val_dataset,
                                      region, "CNN_LSTM", config,
                                      device,
                                      log_suffix=f"K={config['K']}_lr={config['learning_rate']}_p={config['dropout_prob']}_hidden_size={config['hidden_size']}")
  # pred_y = np.concatenate(total_preds, axis=0)
  # val_y = np.concatenate([y for x, y in val_dataset])

  # pred_list.append((pred_y, val_y))
  # loss_list.append(losses)

/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
 19%|█▉        | 96/500 [02:35<10:52,  1.62s/it]


In [20]:
config = {
    "learning_rate": 1e-4,
    "epochs": 500,
    "batch_size": 32,
    "num_layers": 2,
    "hidden_size": 128,
    "window_size": 24,
    "output_size": 1,
    "dropout_prob": 0.4,
    "patience": 10,
    "out_channels": 6,
    "kernel_size": 3,
    "K": 1,
}

regions = ["Gwangju"]

pred_list, loss_list = [], []

for region in regions:
  if region == "Jeonju":
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "IX", "PS", "지점", "위도", "경도"]
  else:
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]
  df = prepare_data_for_CNN_LSTM(region.lower(), columns_to_remove)
  df = simple_interpolate(df, method="linear")
  df, scaler = minmax_scaling(df)

  dataset = FinedustCNNLSTMDataset(df,
                                  window_size=config["window_size"],
                                  prediction_length=config["output_size"],
                                  time_window=3)
  train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
  train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

  model = FinedustCNNLSTM(config,
                          in_channels=len(dataset.pm_columns),
                          input_size=len(dataset.feature_columns)).to(device)

  total_preds, losses = train_cnn_lstm(model,
                                      train_dataset,
                                      val_dataset,
                                      region, "CNN_LSTM", config,
                                      device,
                                      log_suffix=f"K={config['K']}_lr={config['learning_rate']}_p={config['dropout_prob']}")
  # pred_y = np.concatenate(total_preds, axis=0)
  # val_y = np.concatenate([y for x, y in val_dataset])

  # pred_list.append((pred_y, val_y))
  # loss_list.append(losses)

/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
  3%|▎         | 14/500 [00:16<09:43,  1.20s/it]


In [13]:
regions = ["Andong", "Seoul", "Jeonju", "Daegu", "Gwangju"]

pred_list, loss_list = [], []

for region in regions:
  if region == "Jeonju":
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "IX", "PS", "지점", "위도", "경도"]
  else:
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]
  df = prepare_data_for_CNN_LSTM(region.lower(), columns_to_remove)
  df = simple_interpolate(df, method="linear")
  df, scaler = minmax_scaling(df)

  dataset = FinedustCNNLSTMDataset(df,
                                  window_size=config["window_size"],
                                  prediction_length=config["output_size"],
                                  time_window=3)
  train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
  train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

  model = FinedustCNNLSTM(config,
                          in_channels=len(dataset.pm_columns),
                          input_size=len(dataset.feature_columns)).to(device)

  total_preds, losses = train_cnn_lstm(model,
                                      train_dataset,
                                      val_dataset,
                                      region, "CNN_LSTM", config,
                                      device,
                                      log_suffix=f"K={config['K']}_lr={config['learning_rate']}_p={config['dropout_prob']}")
  # pred_y = np.concatenate(total_preds, axis=0)
  # val_y = np.concatenate([y for x, y in val_dataset])

  # pred_list.append((pred_y, val_y))
  # loss_list.append(losses)

/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
 35%|███▍      | 173/500 [03:33<06:43,  1.23s/it]
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in 

## Extract important features

In [7]:
regions

['Andong', 'Seoul', 'Jeonju', 'Daegu', 'Gwangju']

In [24]:
region

model_name[region]

'Gwangju_K=1.pth'

In [27]:
configurations = {"Seoul": {
    "learning_rate": 1e-4,
    "epochs": 500,
    "batch_size": 32,
    "num_layers": 2,
    "hidden_size": 256,
    "window_size": 24,
    "output_size": 1,
    "dropout_prob": 0.2,
    "patience": 10,
    "out_channels": 6,
    "kernel_size": 3,
    "K": 2,
    },
 "Jeonju": {
     'learning_rate': 0.0001,
     'epochs': 500,
     'batch_size': 32,
     'num_layers': 2,
     'hidden_size': 128,
     'window_size': 24,
     'output_size': 1,
     'dropout_prob': 0.2,
     'patience': 10,
     'out_channels': 4,
     'kernel_size': 3,
     'K': 2
     },
 "Gwangju": {
     'learning_rate': 0.0002,
     'epochs': 500,
     'batch_size': 32,
     'num_layers': 2,
     'hidden_size': 128,
     'window_size': 24,
     'output_size': 1,
     'dropout_prob': 0.2,
     'patience': 10,
     'out_channels': 4,
     'kernel_size': 3,
     'K': 1
     },
 "Daegu": {
     'learning_rate': 0.0002,
     'epochs': 500,
     'batch_size': 32,
     'num_layers': 2,
     'hidden_size': 128,
     'window_size': 24,
     'output_size': 1,
     'dropout_prob': 0.2,
     'patience': 10,
     'out_channels': 4,
     'kernel_size': 3,
     'K': 1
     },
 "Andong": {
     'learning_rate': 0.0002,
     'epochs': 500,
     'batch_size': 32,
     'num_layers': 2,
     'hidden_size': 128,
     'window_size': 24,
     'output_size': 1,
     'dropout_prob': 0.2,
     'patience': 10,
     'out_channels': 4,
     'kernel_size': 3,
     'K': 1
     },
}

In [29]:
model_name = "CNN_LSTM"

detailed_model_name = {
    "Jeonju": "Jeonju_K=2_lr=0.0001.pth",
    "Seoul": "Seoul_K=2_lr=0.0001_p=0.2_hidden_size=256.pth",
    "Gwangju": "Gwangju_K=1.pth",
    "Daegu": "Daegu_K=1.pth",
    "Andong": "Andong_K=1.pth",
}

output_dir = f"importance/{model_name}"
os.makedirs(output_dir, exist_ok=True)

regions = ["Andong", "Seoul", "Jeonju", "Daegu", "Gwangju"]
for region in regions:
  if region == "Jeonju":
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "IX", "PS", "지점", "위도", "경도"]
  else:
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]
  df = prepare_data_for_CNN_LSTM(region.lower(), columns_to_remove)
  df = simple_interpolate(df, method="linear")
  df, scaler = minmax_scaling(df)

  dataset = FinedustCNNLSTMDataset(df,
                                 window_size=config["window_size"],
                                 prediction_length=config["output_size"],
                                 time_window=3)
  train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
  train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

  model = FinedustCNNLSTM(configurations[region],
                          in_channels=len(dataset.pm_columns),
                          input_size=len(dataset.feature_columns)).to(device)
  model_path = detailed_model_name[region]
  save_path = f"models/{model_name}/{model_path}"
  model.load_state_dict(torch.load(save_path))
  model.to(device)

  feature_importances = compute_cnn_lstm_permutation_importance(model, val_loader, dataset.feature_columns, dataset.pm_columns, device)
  output_path = f"{output_dir}/{region}.txt"

  with open(output_path, "w") as f:
    f.write("Feature Importances:\n")
    for feature, importance in sorted(feature_importances.items(), key=lambda x: x[1], reverse=True):
        f.write(f"{feature}: {importance:.4f}\n")
  print(f"Feature importances saved to {output_path}")

/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
<ipython-input-29-75d1790557e9>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `Tr

Baseline MSE Loss: 9.1302
Feature: WD, Permutation Importance: 2.3762
Feature: WS, Permutation Importance: 1.5686
Feature: TA, Permutation Importance: 3.2195
Feature: TD, Permutation Importance: 2.2089
Feature: HM, Permutation Importance: 2.2627
Feature: PV, Permutation Importance: 3.9285
Feature: VS, Permutation Importance: 5.6394
Feature: TS, Permutation Importance: 0.6916
Feature: PM10_yanan, Permutation Importance: 0.0023
Feature: PM10_qingdao, Permutation Importance: -0.0037
Feature: PM10_chifeng, Permutation Importance: -0.0002
Feature: PM10_dalian, Permutation Importance: 2.1502
Feature importances saved to importance/CNN_LSTM/Andong.txt


/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
<ipython-input-29-75d1790557e9>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `Tr

Baseline MSE Loss: 18.1466
Feature: WD, Permutation Importance: 1.2138
Feature: WS, Permutation Importance: 0.7677
Feature: TA, Permutation Importance: 1.0971
Feature: TD, Permutation Importance: 1.1031
Feature: HM, Permutation Importance: 1.1430
Feature: PV, Permutation Importance: 0.7095
Feature: VS, Permutation Importance: 2.5766
Feature: TS, Permutation Importance: 0.5012
Feature: TE_005, Permutation Importance: 0.3030
Feature: TE_01, Permutation Importance: 0.1877
Feature: TE_02, Permutation Importance: 0.4381
Feature: TE_03, Permutation Importance: 0.1836
Feature: PM10_yanan, Permutation Importance: 0.0486
Feature: PM10_qingdao, Permutation Importance: -0.0005
Feature: PM10_chifeng, Permutation Importance: 0.0078
Feature: PM10_dalian, Permutation Importance: 2.4788
Feature importances saved to importance/CNN_LSTM/Seoul.txt


/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
<ipython-input-29-75d1790557e9>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `Tr

Baseline MSE Loss: 11.3693
Feature: WD, Permutation Importance: 3.0817
Feature: WS, Permutation Importance: 1.2251
Feature: TA, Permutation Importance: 1.3840
Feature: TD, Permutation Importance: 0.5194
Feature: HM, Permutation Importance: 4.4676
Feature: PV, Permutation Importance: 2.3970
Feature: VS, Permutation Importance: 5.4669
Feature: TS, Permutation Importance: 1.7910
Feature: PM10_yanan, Permutation Importance: 0.0077
Feature: PM10_qingdao, Permutation Importance: 0.0201
Feature: PM10_chifeng, Permutation Importance: 0.0344
Feature: PM10_dalian, Permutation Importance: 2.2599
Feature importances saved to importance/CNN_LSTM/Jeonju.txt


/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
<ipython-input-29-75d1790557e9>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `Tr

Baseline MSE Loss: 9.7039
Feature: WD, Permutation Importance: 2.5978
Feature: WS, Permutation Importance: 2.2893
Feature: TA, Permutation Importance: 2.0584
Feature: TD, Permutation Importance: 1.7961
Feature: HM, Permutation Importance: 1.9200
Feature: PV, Permutation Importance: 4.3456
Feature: VS, Permutation Importance: 3.0684
Feature: TS, Permutation Importance: 2.2943
Feature: PM10_yanan, Permutation Importance: 0.2314
Feature: PM10_qingdao, Permutation Importance: 0.0080
Feature: PM10_chifeng, Permutation Importance: 0.0087
Feature: PM10_dalian, Permutation Importance: 2.7178
Feature importances saved to importance/CNN_LSTM/Daegu.txt


/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/content/drive/MyDrive/Colab Notebooks/인지프/AI_Project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
<ipython-input-29-75d1790557e9>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `Tr

Baseline MSE Loss: 16.8140
Feature: WD, Permutation Importance: 1.6599
Feature: WS, Permutation Importance: 1.9550
Feature: TA, Permutation Importance: 2.1144
Feature: TD, Permutation Importance: 1.3323
Feature: HM, Permutation Importance: 3.0487
Feature: PV, Permutation Importance: 2.9689
Feature: VS, Permutation Importance: 3.9445
Feature: TS, Permutation Importance: 2.0069
Feature: PM10_yanan, Permutation Importance: 0.0379
Feature: PM10_qingdao, Permutation Importance: 0.0092
Feature: PM10_chifeng, Permutation Importance: 0.0391
Feature: PM10_dalian, Permutation Importance: 1.7602
Feature importances saved to importance/CNN_LSTM/Gwangju.txt
